In [218]:
import numpy as np
import pandas as pd
import string
import warnings
warnings.filterwarnings('ignore')

In [219]:
emails= pd.read_excel('emails.xlsx')
print(emails.columns)
print("---------")
print(emails.shape)

Index(['hasAttachments', 'sender name', 'sender email', 'subject', 'body'], dtype='object')
---------
(1063, 5)


In [220]:
# extract domain of email
temp_df = emails['sender email'].str.split('@', n = 1, expand = True)
emails['sender email domain']= temp_df[1]
emails['sender email domain'].replace('[.]\w+$', '', inplace=True, regex=True)
del emails['sender email']
print(emails.columns)

Index(['hasAttachments', 'sender name', 'subject', 'body',
       'sender email domain'],
      dtype='object')


In [221]:
# lower case subject and body columns
emails['sender name']= emails['sender name'].str.lower()
emails['subject']= emails['subject'].str.lower()
emails['body']= emails['body'].str.lower()

In [222]:
# replace hasAttachments NAN to 0
emails['hasAttachments'] = emails['hasAttachments'].fillna(0)

In [223]:
# remove replied emails
print(emails.shape)
print("----------")
emails = emails[~emails['subject'].astype(str).str.startswith('re:')]
emails = emails[~emails['subject'].astype(str).str.startswith('re :')]
print(emails.shape)

(1063, 5)
----------
(720, 5)


In [224]:
# remove the 'frw:' in subject
emails['subject'] = emails['subject'].str.replace('fwd:','')
emails['subject'] = emails['subject'].str.replace('fw:','')

In [225]:
emails_v1= emails.copy()

In [226]:
print(emails_v1['body'][0])

[cid:image019.jpg@01d6ca5c.7d4a1f60]

chères étudiantes, chers étudiants,

dans le cadre du programme « open minds », nous sommes heureux de vous convier à notre prochaine conférence -en direct- le mercredi 09 décembre 2020 de 19h00 à 20h30 sur le thème « femmes, hommes, société des inégaux, société des vulnérables ». animé par pr. najat vallaud belkacem, le débat sera l’occasion d’aborder les différents aspects du sujet proposé.

souhaitez-vous poser une question en direct à pr. najat vallaud belkacem?

si oui, nous vous invitons à remplir le formulaire sur le lien suivant : https://bit.ly/posez_une_question_prnajat

pour vous inscrire, nous vous remercions de remplir le formulaire d’inscription sur le lien suivant : https://bit.ly/openminds2

inscription ouverte à tous les étudiants um6p.
langue: français.


[cid:image020.jpg@01d6ca5c.7d4a1f60]

[pour s’inscrire]<https://bit.ly/openminds2>      [pour plus d’informations] <https://bit.ly/flyer_najat_vallaud_belkacem>       [votre cour

In [227]:
#remove unwanted patterns in forwarded messages
emails_v1['body'].replace('from:.*\nsent:.*\nto:.*\n(.*\n)*subject:.*', '', inplace=True, regex=True)
emails_v1['body'].replace('from:.*\ndate:.*\nsubject:.*\n(.*\n)*to:.*\n', '', inplace=True, regex=True)
emails_v1['body'].replace('de\s?:.*\nenvoyé\s?:.*\nà\s?:.*\n(.*\n)*objet\s?:.*', '', inplace=True, regex=True)
emails_v1['body'].replace('caution: this email originat.+content is safe.', '', inplace=True, regex=True)

In [228]:
#remove: links and emails
emails_v1['body'].replace('http\S+', ' ', inplace=True, regex=True)
emails_v1['body'].replace('www\.\S+\.\S+', ' ', inplace=True, regex=True)
emails_v1['body'].replace('\S+@\S+', ' ', inplace=True, regex=True)

In [229]:
#Expand Abbreviation
#replace tp, tp1, tp2, tp3 ===> travaux pratiques
emails_v1['subject'].replace('tp[0-9]*', 'travaux pratiques', inplace=True, regex=True)
emails_v1['body'].replace('tp[0-9]*', 'travaux pratiques', inplace=True, regex=True)
#replace tds, td1, td2 ===> travaux dirigés
emails_v1['subject'].replace('td[0-9]+', 'travaux dirigés', inplace=True, regex=True)
emails_v1['body'].replace('td[0-9]+', 'travaux dirigés', inplace=True, regex=True)
#replace s1, s2, s3 ===> semestre
emails_v1['subject'].replace('s[0-9]+', 'semestre', inplace=True, regex=True)
emails_v1['body'].replace('s[0-9]+', 'semestre', inplace=True, regex=True)
#replace um6p ===> université polytechnique
emails_v1['subject'].replace('um[0-9]+p', 'université polytechnique', inplace=True, regex=True)
emails_v1['body'].replace('um[0-9]+p', 'université polytechnique', inplace=True, regex=True)

#remove numbers
emails_v1['subject'].replace('\d+', '', inplace=True, regex=True)
emails_v1['body'].replace('\d+', '', inplace=True, regex=True)

# remove empty lines
for index, row in emails_v1[emails_v1['body'].notna()]['body'].iteritems():
    emails_v1['body'][index] = "".join([s for s in row.strip().splitlines(True) if s.strip()])
    
#remove punctuation
emails_v1['body']=emails_v1['body'].str.replace('[{}]'.format(string.punctuation), ' ')
emails_v1['subject']=emails_v1['subject'].str.replace('[{}]'.format(string.punctuation), ' ')

#remove leading and ending spaces
emails_v1['subject'] = emails_v1['subject'].str.strip()
emails_v1['body'] = emails_v1['body'].str.strip()
#remove duplicated spaces
emails_v1['subject'] = emails_v1[emails_v1['subject'].notna()]['subject'].apply(lambda row: " ".join(row.split())) 
emails_v1['body'] = emails_v1[emails_v1['body'].notna()]['body'].apply(lambda row: " ".join(row.split()))

In [230]:
print(emails_v1['body'][0])

chères étudiantes chers étudiants dans le cadre du programme « open minds » nous sommes heureux de vous convier à notre prochaine conférence en direct le mercredi décembre de h à h sur le thème « femmes hommes société des inégaux société des vulnérables » animé par pr najat vallaud belkacem le débat sera l’occasion d’aborder les différents aspects du sujet proposé souhaitez vous poser une question en direct à pr najat vallaud belkacem si oui nous vous invitons à remplir le formulaire sur le lien suivant pour vous inscrire nous vous remercions de remplir le formulaire d’inscription sur le lien suivant inscription ouverte à tous les étudiants université polytechnique langue français pour s’inscrire pour plus d’informations votre cours sur canvas pour toute information complémentaire et renseignement n hésitez pas à nous contacter par e mail sur «open minds» organizing committee copyright © université polytechnique all rights reserved université mohammed vi polytechnique hay moulay rachid

In [231]:
#detecter la langue
emails_v2= emails_v1.copy()
from langdetect import detect
for index, row in emails_v1['body'].iteritems():
    try:
        language = detect(row)
    except:
        language = "error"
    emails_v2.loc[index, 'language'] = language

In [232]:
emails_v2['language'].value_counts()

fr       397
en       279
error     34
es         2
af         2
it         2
id         1
ca         1
cy         1
hu         1
Name: language, dtype: int64

In [233]:
emails_v2[emails_v2['language'] == 'error']

,hasAttachments,sender name,subject,body,sender email domain,language
9,1.0,khaoula abdenouri,,,um6p,error
17,1.0,khaoula abdenouri,web mining atelier,,um6p,error
32,1.0,mehdi abouhane,travaux pratiques deeplearning,,um6p,error
40,1.0,khaoula abdenouri,,,um6p,error
41,1.0,khaoula abdenouri,cours travaux pratiques replication,,um6p,error
54,0.0,khaoula abdenouri,atelier,,um6p,error
320,1.0,khaoula abdenouri,notebook à rempalcer,,um6p,error
321,1.0,khaoula abdenouri,atelier,,um6p,error
322,1.0,khaoula abdenouri,cours de text mining,,um6p,error
356,1.0,khaoula abdenouri,,,um6p,error


In [234]:
# Replace error body values with Nan
emails_v2.loc[emails_v2['language'] == 'error', 'body'] = np.nan
# replace empty subject with Nan
emails_v2.loc[emails_v2['subject'] == ' ', 'subject'] = np.nan
emails_v2[emails_v2['language'] == 'error']
#remove rows with body and subject nan
emails_v2= emails_v2.drop(emails_v2[emails_v2['body'].isnull() & emails_v2['subject'].isnull()].index)
emails_v2[emails_v2['language'] == 'error']

,hasAttachments,sender name,subject,body,sender email domain,language
9,1.0,khaoula abdenouri,,NaN,um6p,error
17,1.0,khaoula abdenouri,web mining atelier,NaN,um6p,error
32,1.0,mehdi abouhane,travaux pratiques deeplearning,NaN,um6p,error
40,1.0,khaoula abdenouri,,NaN,um6p,error
41,1.0,khaoula abdenouri,cours travaux pratiques replication,NaN,um6p,error
54,0.0,khaoula abdenouri,atelier,NaN,um6p,error
320,1.0,khaoula abdenouri,notebook à rempalcer,NaN,um6p,error
321,1.0,khaoula abdenouri,atelier,NaN,um6p,error
322,1.0,khaoula abdenouri,cours de text mining,NaN,um6p,error
356,1.0,khaoula abdenouri,,NaN,um6p,error


In [235]:
#detect language in subject
emails_v3= emails_v2.copy()
df_subject_with_errors= emails_v2[emails_v2['language'] == "error"]
for index, row in df_subject_with_errors['subject'].iteritems():
    try:
        language = detect(row)
    except:
        language = "error"
    emails_v3.loc[index, 'language'] = language

In [236]:
emails_v3['language'].value_counts()

fr       407
en       280
error      7
af         5
es         2
sw         2
it         2
id         1
ca         1
pt         1
sk         1
cy         1
hu         1
Name: language, dtype: int64

In [237]:
emails_v3.head()

,hasAttachments,sender name,subject,body,sender email domain,language
0,0.0,um6p events,open minds conférence pr najat vallaud belkacem,chères étudiantes chers étudiants dans le cadr...,um6p,fr
1,1.0,sportsbooking,les modifications du rendez vous ont été enreg...,mohammed vi polytechnic university pôle sport ...,um6p,fr
2,0.0,hassan machkour,msda al khwarizmi cours doctoraux seconde edit...,le département msda al khwarizmi modeling simu...,um6p,en
3,0.0,nabila idar,al khwarizmi planning de la semaine du au déce...,bonjour merci de trouver ci dessous le plannin...,um6p,fr
4,1.0,sportsbooking,votre rendez vous a été confirmé avec succès,mohammed vi polytechnic university pôle sport ...,um6p,fr


In [238]:
#export english version
emails_french = emails_v3[emails_v3['language'] == "fr"]
emails_english = emails_v3[emails_v3['language'] == "en"]
emails_french.to_html("emails_cleaned_french.html", encoding='utf-8', index=False)
emails_english.to_html("emails_cleaned_english.html", encoding='utf-8', index=False)
emails_french.to_excel("emails_cleaned_french.xlsx", index=False)
emails_english.to_excel("emails_cleaned_english.xlsx", index=False)